In [2]:
import pandas as pd
from dns.resolver import resolve, resolve_at
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import thread_map, process_map
tqdm.pandas()
# From https://host.io/rankings
#df = pd.read_csv("host10m.csv.gz", names=["domain", "rank"]).head(100000)
#df

In [12]:
blocked_domains = pd.read_csv("blocked_domains", header=None, names=["domain"])
blocked_domains["response"] = blocked_domains.domain.progress_apply(resolve)
blocked_domains

  0%|          | 0/18 [00:00<?, ?it/s]

,domain,response
0,keystone-akamai.mwbsys.com,(205.185.208.98)
1,keystone.mwbsys.com,"(18.207.69.151, 44.218.130.198, 34.203.176.211..."
2,links.malwarebytes.com,"(18.155.202.36, 18.155.202.69, 18.155.202.111,..."
3,meps.mwbsys.com,"(65.9.141.29, 65.9.141.129, 65.9.141.92, 65.9...."
4,nebula-agent-installers-mb-prod.s3.amazonaws.com,"(52.217.203.97, 52.217.192.97, 3.5.28.73, 52.2..."
5,sirius.mwbsys.com,"(54.174.86.67, 54.235.213.30, 35.174.199.124, ..."
6,socket.cloud.malwarebytes.com,"(54.81.70.227, 44.219.222.165, 52.72.175.157, ..."
7,storage.gra.cloud.ovh.net,"(54.38.230.75, 87.98.187.201)"
8,telemetry.malwarebytes.com,"(44.236.24.179, 54.148.201.44, 35.81.231.225, ..."
9,ark.mwbsys.com,"(44.194.11.111, 44.199.68.15, 34.231.228.179)"


In [15]:
blocked_ips = set()
for resp in blocked_domains.response:
    blocked_ips.update([str(r) for r in resp])
len(blocked_ips)

73

In [21]:
def calc_failure_rate(response):
    if response:
        return sum([str(r) in blocked_ips for r in response])/len(response)

def safe_resolve(domain):
  try:
    return resolve(domain)
  except Exception as e:
    print(e)

df = pd.read_csv("domains_to_check", header=None, names=["domain"])
#df["response"] = df.domain.progress_apply(resolve)
df["response"] = thread_map(safe_resolve, df.domain)
df["failure_rate"] = df.response.apply(calc_failure_rate)
df.sort_values("failure_rate")

  0%|          | 0/9 [00:00<?, ?it/s]

,domain,response,failure_rate
2,files.slack.com,"(65.9.141.8, 65.9.141.60, 65.9.141.27, 65.9.14...",0.00
4,video.alexanderstreet.com,"(65.9.141.28, 65.9.141.124, 65.9.141.31, 65.9....",0.00
6,www.emerald.com,"(3.163.189.39, 3.163.189.114, 3.163.189.47, 3....",0.00
7,www.degruyter.com,"(108.157.254.103, 108.157.254.80, 108.157.254....",0.00
8,reports.hitsongsdeconstructed.com,(67.227.227.178),0.00
0,cdn.evbstatic.com,"(65.9.141.56, 65.9.141.57, 65.9.141.112, 65.9....",0.25
5,search.credoreference.com,"(65.9.141.100, 65.9.141.45, 65.9.141.11, 65.9....",0.25
1,metromap.com.au,"(65.9.141.27, 65.9.141.128, 65.9.141.129, 65.9...",0.50
3,scite.ai,"(65.9.141.106, 65.9.141.86, 65.9.141.14, 65.9....",0.50


In [ ]:
def ans_to_str(response):
  if response:
    return [str(a) for a in response]
df.response = df.response.apply(ans_to_str)
df.to_csv("100K_domains_resolved.csv", index=False)